# REDES NEURAIS ARTIFICIAIS

[Aula 3 de Deep Learning](http://bit.ly/dn-unb03) da Engenharia de Software da UnB

# Parte 1 - Pré-processamento dos Dados

### Importar as libs

In [18]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [19]:
tf.__version__

'2.4.1'

### Importar o dataset

In [20]:
dataset = pd.read_csv('Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [21]:
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values
print("X >>\n",X)
print("y >>\n",y)

X >>
 [[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
y >>
 [1 0 1 ... 1 1 0]


### Transformando os dados categóricos


In [22]:
# Label Encoding the "Gender" column
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])
print(X)
# One Hot Encoding the "Geography" column
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]
[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


# Dividindo o dataset em conjunto de treinamento e conjunto de teste

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling

In [24]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [25]:
np.shape(X_train)

(8000, 12)

---
# Parte 2 -Vamos construir uma ANN!


### Initializing the ANN

In [26]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [27]:
ann.add(tf.keras.layers.Dense(units=5, activation='tanh'))

### Adding the second hidden layer

In [28]:
ann.add(tf.keras.layers.Dense(units=5, activation='tanh'))

### Adding the output layer

In [29]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [30]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [31]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 200)

Epoch 1/200
250/250 [==============================] - 1s 2ms/step - loss: 0.6887 - accuracy: 0.5805
Epoch 2/200
250/250 [==============================] - 0s 2ms/step - loss: 0.5194 - accuracy: 0.8006
Epoch 3/200
250/250 [==============================] - 0s 2ms/step - loss: 0.4581 - accuracy: 0.7925
Epoch 4/200
250/250 [==============================] - 0s 1ms/step - loss: 0.4346 - accuracy: 0.7980
Epoch 5/200
250/250 [==============================] - 0s 965us/step - loss: 0.4204 - accuracy: 0.8004
Epoch 6/200
250/250 [==============================] - 0s 2ms/step - loss: 0.4256 - accuracy: 0.8023
Epoch 7/200
250/250 [==============================] - 0s 1ms/step - loss: 0.4191 - accuracy: 0.8062
Epoch 8/200
250/250 [==============================] - 0s 1ms/step - loss: 0.4083 - accuracy: 0.8090
Epoch 9/200
250/250 [==============================] - 0s 2ms/step - loss: 0.4043 - accuracy: 0.8128
Epoch 10/200
250/250 [==============================] - 0s 1ms/step - loss: 0.4011 - accu

# Parte 3 - Fazendo predições e avaliando o modelo

## Prevendo os resultados com o conjunto de testes

In [32]:
y_pred = ann.predict(X_test)
y_pred

array([[0.24286771],
       [0.27160147],
       [0.12734953],
       ...,
       [0.11726439],
       [0.16618916],
       [0.28448045]], dtype=float32)

In [33]:
y_pred = (y_pred > 0.5)
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

# Criando uma Confusion Matrix

In [34]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1511   84]
 [ 192  213]]


# FIM

> Professor Diego Dorgam  
> [@diegodorgam](https://twitter.com/diegodorgam)